In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Summarize messages

In [8]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware

from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model = "gemini-2.5-flash", temperature=0.5)  

agent = create_agent(
    #model="gpt-5-nano",
    model = model,
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            #model="gpt-4o-mini",
            model = model,
            trigger=("tokens", 100),
            keep=("messages", 1)
        )
    ],
)

In [9]:
from langchain.messages import HumanMessage, AIMessage
from pprint import pprint

response = agent.invoke(
    {"messages": [
        HumanMessage(content="What is the capital of the moon?"),
        AIMessage(content="The capital of the moon is Lunapolis."),
        HumanMessage(content="What is the weather in Lunapolis?"),
        AIMessage(content="Skies are clear, with a high of 120C and a low of -100C."),
        HumanMessage(content="How many cheese miners live in Lunapolis?"),
        AIMessage(content="There are 100,000 cheese miners living in Lunapolis."),
        HumanMessage(content="Do you think the cheese miners' union will strike?"),
        AIMessage(content="Yes, because they are unhappy with the new president."),
        HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?"),
        ]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'messages': [HumanMessage(content="Here is a summary of the conversation to date:\n\nThe capital of the moon is Lunapolis. The weather in Lunapolis is clear, with a high of 120C and a low of -100C. There are 100,000 cheese miners living in Lunapolis. The cheese miners' union may strike due to unhappiness with the new president.", additional_kwargs={}, response_metadata={}, id='d692f10b-6211-460a-96af-6fd54716f7cb'),
              HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?", additional_kwargs={}, response_metadata={}, id='792b9c51-3ce1-4a48-9f26-c1a7b8504d33'),
              AIMessage(content='As the new president of Lunapolis, facing a potential strike from our 100,000 cheese miners – who are undoubtedly the backbone of our economy and society, especially given the challenging conditions (120C high, -100C low) – my response would be swift, empathetic, and proactive.\n\nHere\'s my plan:\n\n1.  **Immediate Open Communica

In [10]:
print(response["messages"][0].content)

Here is a summary of the conversation to date:

The capital of the moon is Lunapolis. The weather in Lunapolis is clear, with a high of 120C and a low of -100C. There are 100,000 cheese miners living in Lunapolis. The cheese miners' union may strike due to unhappiness with the new president.


## Trim/delete messages

In [13]:
from typing import Dict, Any
from langchain.agents import AgentState
from langchain.messages import RemoveMessage
from langgraph.runtime import Runtime
from langchain.agents.middleware import before_agent
from langchain.messages import ToolMessage


@before_agent
def trim_messages(state: AgentState, runtime: Runtime) -> Dict[str, Any] | None:
    """Remove all the tool messages from the state"""
    messages = state["messages"]


    tool_messages = [m for m in messages if isinstance(m, ToolMessage)]
    
    return {"messages": [RemoveMessage(id=m.id) for m in tool_messages]}

In [14]:
agent = create_agent(
    #model="gpt-5-nano",
    model = model, 
    checkpointer=InMemorySaver(),
    middleware=[trim_messages],
)

In [15]:
response = agent.invoke(
    {"messages": [
        HumanMessage(content="My device won't turn on. What should I do?"),
        ToolMessage(content="blorp-x7 initiating diagnostic ping…", tool_call_id="1"),
        AIMessage(content="Is the device plugged in and turned on?"),
        HumanMessage(content="Yes, it's plugged in and turned on."),
        ToolMessage(content="temp=42C voltage=2.9v … greeble complete.", tool_call_id="2"),
        AIMessage(content="Is the device showing any lights or indicators?"),
        HumanMessage(content="What's the temperature of the device?")
        ]},
    {"configurable": {"thread_id": "2"}}
)

pprint(response)

{'messages': [HumanMessage(content="My device won't turn on. What should I do?", additional_kwargs={}, response_metadata={}, id='47843dec-c545-4cd5-a60c-83fb89fddfe6'),
              AIMessage(content='Is the device plugged in and turned on?', additional_kwargs={}, response_metadata={}, id='317f4fc3-d02b-4f47-b2ad-056b1792c701'),
              HumanMessage(content="Yes, it's plugged in and turned on.", additional_kwargs={}, response_metadata={}, id='20e4b40e-27bc-4c66-98b8-50a5a9a92615'),
              AIMessage(content='Is the device showing any lights or indicators?', additional_kwargs={}, response_metadata={}, id='1325eb12-614a-4e9d-b6e0-61502883d4e1'),
              HumanMessage(content="What's the temperature of the device?", additional_kwargs={}, response_metadata={}, id='d9e5dd3e-1346-4eff-b027-bfe83cf2a04a'),
              AIMessage(content="As an AI, I don't have access to your physical device and therefore cannot tell you its temperature directly.\n\nHowever, if you're asking

In [16]:
print(response["messages"][-1].content)

As an AI, I don't have access to your physical device and therefore cannot tell you its temperature directly.

However, if you're asking because you suspect temperature might be an issue (e.g., it feels hot or unusually cold), you can check it yourself by carefully touching the device.

Could you tell me what you observe about its temperature, and if you're seeing any other signs or lights? This might help us figure out why it's not turning on.
